In [1]:
# %load ../startup.py
import os,sys
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

os.environ['PYTHONPATH']=os.environ.get('LIB')
if not os.environ.get('LIB') in sys.path: 
    sys.path.insert(0,os.environ.get('LIB'))

DAT_DIR = os.environ.get('DAT_DIR')    

%load_ext autoreload
%autoreload 2
%pylab inline
%matplotlib inline
%load_ext rpy2.ipython

import matplotlib.text as text
import pandas as pd
import numpy as np
import pylab as pl
import scipy as sp
import sys
import rpy2 
import os 
from matplotlib import gridspec
from scipy.interpolate import splev, splrep
import xlwt
import seaborn as sns
from scipy import stats
import rpy2.robjects.packages as rpackages
import seaborn as sns
from statsmodels import robust


from heprnhci.db.mongo import *


from IPython.core.display import display, HTML
display(HTML("<style>.container {width:80% !important;}</style>")) # increase jupyter screen width to 100%
pd.options.display.max_colwidth = 500
pd.set_option('display.precision',2)

HCI=openMongo(db=os.environ.get('MONGO_HCIDB'),host=os.environ.get('MONGO_HOST'),auth=False)


Populating the interactive namespace from numpy and matplotlib


/opt/miniconda3/envs/tiptox/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/opt/miniconda3/envs/tiptox/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


# Goal

Load, process and normalize raw data



# Read raw data

In [2]:
RW0 = pd.read_csv(DAT_DIR+'cyprotex-heprn2-jul-2017-raw-v2.csv')\
        .drop_duplicates()

In [16]:
HCI.chemicals.find(dict(stype={'$exists':0})).count()
HCI.chemicals.update_many(dict(stype={'$exists':0}),
                          {'$set':dict(stype='test chem')})

/share/home/ishah/.conda/envs/py27a/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


### load chemical information

In [17]:
CH0 = pd.DataFrame(list(HCI.chemicals.find({},dict(_id=0))),
                   )
CH0.head()

,QSAR_READY_SMILES,casrn,chem_name,dsstox_sid,found_by,name,sample_id,stype
0,CN(C)N,57-14-7,"1,1-Dimethylhydrazine",DTXSID1020516,Approved Name,"1,1-Dimethylhydrazine",TP0001526G01,chem_test
1,[O-][N+](=O)C1=CC=C(Cl)C=C1,100-00-5,1-Chloro-4-nitrobenzene,DTXSID5020281,Approved Name,1-Chloro-4-nitrobenzene,TP0001526F03,chem_test
2,OC(=O)COC1=C(Cl)C=C(Cl)C=C1,94-75-7,"2,4-Dichlorophenoxyacetic acid",DTXSID0020442,Approved Name,"2,4-Dichlorophenoxyacetic acid",TP0001526H06,chem_test
3,CCCCOCCO,111-76-2,2-Butoxyethanol,DTXSID1024097,Approved Name,2-Butoxyethanol,TP0001526B01,chem_test
4,CC1=C(N)C=C(C=C1)[N+]([O-])=O,99-55-8,2-Methyl-5-nitroaniline,DTXSID4020959,Approved Name,2-Methyl-5-nitroaniline,TP0001526H02,chem_test


In [85]:
#Chems=list(HCI.chemicals.find({},dict(_id=0)))

### Store chemical information

In [18]:
SID=RW0[['sample_id','dsstox_sid']].drop_duplicates().reset_index()

In [87]:
for x in SID.itertuples():
    if x.dsstox_sid==x.dsstox_sid:
        print x.dsstox_sid,x.sample_id
        HCI.chemicals.update_one(dict(dsstox_sid=x.dsstox_sid),
                               {'$set':dict(sample_id=x.sample_id)})

DTXSID0032655 TP0001526B06
DTXSID0023901 TP0001526B07
DTXSID2039336 TP0001526B08
DTXSID2032390 TP0001526B09
DTXSID9032581 TP0001526B10
DTXSID9032329 TP0001526B11
DTXSID3044203 TP0001526C01
DTXSID8034580 TP0001526C02
DTXSID3034872 TP0001526C03
DTXSID7020348 TP0001526C04
DTXSID4022448 TP0001526C05
DTXSID3024235 TP0001526C06
DTXSID8020961 TP0001526C07
DTXSID2032398 TP0001526C08
DTXSID3024239 TP0001526C09
DTXSID5032498 TP0001526C10
DTXSID8024652 TP0001526A01
DTXSID2020006 TP0001526A02
DTXSID0032601 TP0001526A03
DTXSID4024195 TP0001526A04
DTXSID6032354 TP0001526A05
DTXSID4021268 TP0001526A06
DTXSID9032379 TP0001526A07
DTXSID2020420 TP0001526A08
DTXSID8024159 TP0001526A09
DTXSID3034402 TP0001526A10
DTXSID2032500 TP0001526A11
DTXSID1024097 TP0001526B01
DTXSID1020647 TP0001526B02
DTXSID5032365 TP0001526B03
DTXSID8020127 TP0001526B04
DTXSID2021105 TP0001526B05
DTXSID8035180 TP0001526C11
DTXSID3047344 TP0001526D01
DTXSID6034392 TP0001526D02
DTXSID3020336 TP0001526D03
DTXSID8034877 TP0001526D04
D

In [19]:
CHM0=pd.DataFrame(list(HCI.chemicals.find({},dict(_id=0))),
                  columns=['dsstox_sid','sample_id','casrn','chem_name','stype'])\
    .rename(columns=dict(chem_name='name'))
CHM0.head()

,dsstox_sid,sample_id,casrn,name,stype
0,DTXSID1020516,TP0001526G01,57-14-7,"1,1-Dimethylhydrazine",chem_test
1,DTXSID5020281,TP0001526F03,100-00-5,1-Chloro-4-nitrobenzene,chem_test
2,DTXSID0020442,TP0001526H06,94-75-7,"2,4-Dichlorophenoxyacetic acid",chem_test
3,DTXSID1024097,TP0001526B01,111-76-2,2-Butoxyethanol,chem_test
4,DTXSID4020959,TP0001526H02,99-55-8,2-Methyl-5-nitroaniline,chem_test


In [20]:
CHM0.stype.value_counts()

chem_test    88
ctrl+         4
ctrl-         1
Name: stype, dtype: int64

## Name the features 

In [21]:
FN0=['DNATexture','MitoFxnI','LysosomalMass','NuclearSize1','CellNumber1','ERStress',
       'Apoptosis','NuclearSize2','CellNumber2','Steatosis']
FA0 =['DT','MF','LM','NS1','CN1','ES','Ap','NS2','CN2','St']
FN1=['DNATexture','MitoFxnI','LysosomalMass','NuclearSize','CellNumber','ERStress',
       'Apoptosis','NuclearSize','CellNumber','Steatosis']
FA1 =['DT','MF','LM','NS','CN','ES','Ap','NS','CN','St']

#RW0[['channel_name','feature_name']].drop_duplicates().to_dict('records')
HCIFT = RW0[['channel_name','feature_name']].drop_duplicates().reset_index(drop=True)
HCIFT['FN0']=FN0
HCIFT['FN1']=FN1
HCIFT['FA0']=FA0
HCIFT['FA1']=FA1
HCIFT


,channel_name,feature_name,FN0,FN1,FA0,FA1
0,Hoechst - Nuclei,MEAN_ObjectVarIntenCh1,DNATexture,DNATexture,DT,DT
1,TMRE,%HIGH_RingSpotAvgIntenCh2,MitoFxnI,MitoFxnI,MF,MF
2,LTRed TRITC,MEAN_RingSpotAvgIntenCh2,LysosomalMass,LysosomalMass,LM,LM
3,Hoechst focus,MEAN_ObjectSizeCh1,NuclearSize1,NuclearSize,NS1,NS
4,Hoechst focus,SelectedObjectCountPerValidField,CellNumber1,CellNumber,CN1,CN
5,CY3-GADD153,MEAN_CircAvgIntenCh2,ERStress,ERStress,ES,ES
6,Cy5-CYTO C,MEAN_CircAvgIntenCh3,Apoptosis,Apoptosis,Ap,Ap
7,Nuclei,MEAN_ObjectSizeCh1,NuclearSize2,NuclearSize,NS2,NS
8,Nuclei,SelectedObjectCountPerValidField,CellNumber2,CellNumber,CN2,CN
9,LipidTox-Cy5 kernel 3,MEAN_RingSpotAvgAreaCh2,Steatosis,Steatosis,St,St


In [22]:
RW1 = RW0.merge(HCIFT,left_on=['channel_name','feature_name'],
                right_on=['channel_name','feature_name'])

### Fix the reference chemicals ...

In [23]:
CTRL0={'bupivacaine':'BPVC',
       'carbonyl cyanide 3-chlorophenylhydrazone':'CCCP',
       'nimesulide':'NMSL',
       'tamoxifen':'TMXF'}

RW1.replace(dict(sample_id=CTRL0),inplace=True)
#RW1.replace(dict(casrn=CTRL0),inplace=True)


In [24]:
RW1.head().T
len(RW1.sample_id.unique()),len(CHM0.sample_id.unique())

(93, 93)

In [25]:
RW1.sample_id[RW1.sample_id.str[:2]!='TP'].unique()

array(['DMSO', 'BPVC', 'CCCP', 'NMSL', 'TMXF'], dtype=object)

In [26]:
RW2=RW1.drop(['dsstox_sid','name','casrn'],axis=1)\
        .merge(CHM0,left_on='sample_id',right_on='sample_id',how='left')

In [27]:
RW2.stype.value_counts()

chem_test    105600
ctrl+         14400
ctrl-          7200
Name: stype, dtype: int64

In [28]:
RW1.shape,RW2.shape

((127200, 17), (127200, 18))

Fix the DMSO concentration set to 0

In [29]:
RW2.stype[RW2.conc.isna()].unique()

array([u'ctrl-'], dtype=object)

In [30]:
RW2.conc[RW2.conc.isna()]=0

/share/home/ishah/.conda/envs/py27a/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# Store raw data by plate

In [31]:
HCI.heprn_raw.drop()

for (plate_id,timeh),X in RW2.groupby(['plate_id','timeh']):
    print plate_id,timeh
    P0 = dict(plate_id=plate_id,
              timeh=X.timeh.unique()[0],
              assays0=list(X.FN0.unique()),
              assays1=list(X.FN1.unique()),
              assay_abbrv0=list(X.FA0.unique()),
              assay_abbrv1=list(X.FA1.unique()),
              samples =list(X.sample_id.unique()),
              chemicals=list(X.name.unique()),
              dsstox_sids=list(X.dsstox_sid.unique()),
              ctrl=dict(neg=list(set(X.sample_id[X.stype=='ctrl-'])),
                        pos=list(set(X.sample_id[X.stype=='ctrl+']))),
              wells=list(X.to_dict('records'))
             )
    HCI.heprn_raw.insert_one(P0)


HCS-UC-W73_160624150001 24
HCS-UC-W73_160624160002 24
HCS-UC-W73_160625080001 72
HCS-UC-W73_160625080002 72
HCS-UC-W73_160628150001 48
HCS-UC-W73_160629070001 48
HCS-UC-W73_160712160001 1
HCS-UC-W73_160712170001 1
HCS-UC-W73_160712170002 3
HCS-UC-W73_160712180001 3
HCS-UC-W73_160713080001 24
HCS-UC-W73_160713080002 1
HCS-UC-W73_160713090001 1
HCS-UC-W73_160713090002 1
HCS-UC-W73_160713090003 6
HCS-UC-W73_160713100001 3
HCS-UC-W73_160713100002 3
HCS-UC-W73_160713100003 6
HCS-UC-W73_160713110001 24
HCS-UC-W73_160713110002 1
HCS-UC-W73_160713110003 3
HCS-UC-W73_160713120001 6
HCS-UC-W73_160713170001 6
HCS-UC-W73_160713170002 3
HCS-UC-W73_160714170001 1
HCS-UC-W73_160720080001 6
HCS-UC-W73_160720160001 72
HCS-UC-W73_160720170001 48
HCS-UC-W73_160721130002 48
HCS-UC-W73_160721160001 72
HCS-US-W71_160624150001 24
HCS-US-W71_160624190001 24
HCS-US-W71_160625080002 48
HCS-US-W71_160628170001 72
HCS-US-W71_160629070001 48
HCS-US-W71_160706080001 6
HCS-US-W71_160706090001 3
HCS-US-W71_1607060900

In [33]:
HCI.list_collection_names()

[u'heprn_raw', u'chemicals', u'heprn_plates']

## Update features 

In [40]:
Features=pd.DataFrame(
    dict(FN0=['DNATexture','MitoFxnI','LysosomalMass','NuclearSize1','CellNumber1','ERStress',
           'Apoptosis','NuclearSize2','CellNumber2','Steatosis'],
    FA0 =['DT','MF','LM','NS1','CN1','ES','Ap','NS2','CN2','St'],
    FN1=['DNATexture','MitoFxnI','LysosomalMass','NuclearSize','CellNumber','ERStress',
           'Apoptosis','NuclearSize','CellNumber','Steatosis'],
    FA1 =['DT','MF','LM','NS','CN','ES','Ap','NS','CN','St'])
)

In [41]:
HCI.hci_feats.insert_many(Features.to_dict('records'))

In [7]:
Features

,FA0,FA1,FN0,FN1
0,DT,DT,DNATexture,DNATexture
1,MF,MF,MitoFxnI,MitoFxnI
2,LM,LM,LysosomalMass,LysosomalMass
3,NS1,NS,NuclearSize1,NuclearSize
4,CN1,CN,CellNumber1,CellNumber
5,ES,ES,ERStress,ERStress
6,Ap,Ap,Apoptosis,Apoptosis
7,NS2,NS,NuclearSize2,NuclearSize
8,CN2,CN,CellNumber2,CellNumber
9,St,St,Steatosis,Steatosis
